# Chapter: 16
## Section: Implementing AES encryption in Python

In [ ]:
!pip install pycryptodome
!pip install pycryptodomex

The following code generates a new AES128 key and encrypts a piece of data into a file. We use the EAX mode because it allows the receiver to detect any unauthorized modification (similarly, we could have used other authenticated encryption modes like GCM, CCM or SIV).
(source: https://www.pycryptodome.org/src/examples#encrypt-data-with-aes)

In [2]:
from Cryptodome.Cipher import AES
from Cryptodome.Random import get_random_bytes

# SMILES of Paracetamol (traded name: Tylenol)
data = b'CC(=O)NC1=CC=C(C=C1)O'

key_random = get_random_bytes(16)
cipher_molecule = AES.new(key_random, AES.MODE_EAX)
ciphertext, tag = cipher_molecule.encrypt_and_digest(data)

out_encrypt = open("molecule_enc.bin", "wb")
[out_encrypt.write(x) for x in (cipher_molecule.nonce, tag, ciphertext) ]
out_encrypt.close()

At the other end, the receiver can securely load the piece of data back (if they know the key!). Note that the code generates a ValueError exception when tampering is detected. (source: https://www.pycryptodome.org/src/examples#encrypt-data-with-aes)

In [3]:
in_encrypt = open("molecule_enc.bin", "rb")
nonce, tag, ciphertext = [in_encrypt.read(x) for x in (16, 16, -1) ]
in_encrypt.close()

print('Encrypted data (Ciphertext): {}'.format(ciphertext))
# let's assume that the key is somehow available again
decipher_molecule = AES.new(key_random, AES.MODE_EAX, nonce)
data = decipher_molecule.decrypt_and_verify(ciphertext, tag)
print('Decrypted data: {}'.format(data))

Encrypted data (Ciphertext): b'.\xf3/8ZQ\xd7\xff%/\xb8\xef\xa3\xc6\x17\x94\xa2A\xe5\xe4\x8a'
Decrypted data: b'CC(=O)NC1=CC=C(C=C1)O'
